In [1]:
#

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import MobileNetV2, MobileNet, NASNetMobile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
# 設置數據生成器
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    #brightness_range=[0.9, 1.1],
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
)

# 訓練數據生成器
train_generator = train_datagen.flow_from_directory(
    r'E:\Codes\CV train\回收系統dataset\Garbage',  # 替換為您的資料路徑
    target_size=(224,224),
    batch_size=16,
    class_mode='categorical',
)


validation_datagen = ImageDataGenerator(
    rescale=1./255,
)
# 驗證數據生成器
validation_generator = validation_datagen.flow_from_directory(
    r'E:\Codes\CV train\回收系統dataset\TrashBox',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
)

Found 8079 images belonging to 4 classes.
Found 9663 images belonging to 4 classes.


In [5]:
# 加載預訓練模型
base_model = MobileNetV2(
    alpha=0.35,
    weights='imagenet', 
    include_top=False, 
    input_shape=(224, 224, 3))

# 添加自定義分類層
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(64,activation='relu')(x)
x = Dense(4, activation='softmax')(x)

# 構建完整模型
model = Model(inputs=base_model.input, outputs=x)

# 解凍預訓練層
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[:4]:  
    layer.trainable = True

    
# 編譯模型
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [6]:
checkpoint = ModelCheckpoint(
    "test.h5",   # 保存的文件名稱
    monitor="val_accuracy",  # 監測驗證準確率（也可以改成 "val_loss"）
    save_best_only=True,     # 只儲存最佳權重
    mode="max",              # "max" 代表數值越高越好（適用於準確率）
    verbose=1                # 顯示存檔訊息
)

In [7]:
# 訓練模型
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30,
    steps_per_epoch=train_generator.samples // 16,
    validation_steps=validation_generator.samples // 16,
    callbacks=[checkpoint]  
)


Epoch 1/30
504/504 [==============================] - 66s 126ms/step - loss: 0.3848 - accuracy: 0.8453 - val_loss: 0.8874 - val_accuracy: 0.6578

Epoch 00001: val_accuracy improved from -inf to 0.65775, saving model to test.h5
Epoch 2/30


c:\Users\willy\anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


504/504 [==============================] - 62s 123ms/step - loss: 0.2557 - accuracy: 0.9004 - val_loss: 0.9751 - val_accuracy: 0.6815

Epoch 00002: val_accuracy improved from 0.65775 to 0.68149, saving model to test.h5
Epoch 3/30
504/504 [==============================] - 62s 123ms/step - loss: 0.2227 - accuracy: 0.9139 - val_loss: 1.0189 - val_accuracy: 0.6857

Epoch 00003: val_accuracy improved from 0.68149 to 0.68574, saving model to test.h5
Epoch 4/30
504/504 [==============================] - 62s 123ms/step - loss: 0.2033 - accuracy: 0.9222 - val_loss: 1.0453 - val_accuracy: 0.6620

Epoch 00004: val_accuracy did not improve from 0.68574
Epoch 5/30
504/504 [==============================] - 63s 126ms/step - loss: 0.1834 - accuracy: 0.9286 - val_loss: 1.3062 - val_accuracy: 0.6730

Epoch 00005: val_accuracy did not improve from 0.68574
Epoch 6/30
504/504 [==============================] - 62s 124ms/step - loss: 0.1727 - accuracy: 0.9343 - val_loss: 1.6052 - val_accuracy: 0.6347

Epo

KeyboardInterrupt: 

In [ ]:
#final_model: 7385
#VGG16: 8702
#VGG19: 8686
#ResNet50: 5409
#ResNet101: 5729
#ResNet152: 5881
#MobileNet: 9319
#MobileNetV2: 9223
#MobileNetV3Large: 5208
#MobileNetV3Small: 5304
#DenseNet201: 9407
#Xception: 9407
#NASNetMobile: 9183
#NASNetLarge: 9439
#EfficientNetB0: 5240
#EfficientNetB4: 5224
#EfficientNetB7: 5232
#InceptionResNetV2: 9327
#InceptionV3: 9215
